In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from vamos import (
    optimize, OptimizeConfig, NSGAIIConfig, ZDT1,
    weighted_sum_scores, tchebycheff_scores,
    reference_point_scores, knee_point_scores,
)

plt.style.use("ggplot")
print("MCDM methods loaded!")

## 1. Generate a Pareto Front

In [ ]:
# Run optimization to get a Pareto front
zdt1 = ZDT1(n_var=30)

config = NSGAIIConfig().pop_size(100).engine("numpy").fixed()

result = optimize(OptimizeConfig(
    problem=zdt1,
    algorithm="nsgaii",
    algorithm_config=config,
    termination=("n_eval", 10000),
    seed=42,
))

F = result.F  # Pareto front
print(f"Pareto front: {F.shape[0]} solutions, {F.shape[1]} objectives")

In [ ]:
# Visualize the front
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(F[:, 0], F[:, 1], s=40, alpha=0.7, c='steelblue')
ax.set_xlabel("f1 (minimize)")
ax.set_ylabel("f2 (minimize)")
ax.set_title("ZDT1 Pareto Front - Which Solution to Choose?")
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 2. Weighted Sum Method

Assigns importance weights to each objective and selects the solution minimizing the weighted sum:

$$\text{score} = w_1 \cdot f_1 + w_2 \cdot f_2 + \ldots$$

**Use when:** You know the relative importance of objectives.

In [ ]:
# Equal weights: both objectives equally important
weights_equal = np.array([0.5, 0.5])
ws_equal = weighted_sum_scores(F, weights_equal)

# f1-focused: prioritize minimizing f1
weights_f1 = np.array([0.8, 0.2])
ws_f1 = weighted_sum_scores(F, weights_f1)

# f2-focused: prioritize minimizing f2
weights_f2 = np.array([0.2, 0.8])
ws_f2 = weighted_sum_scores(F, weights_f2)

print("Weighted Sum Results:")
print(f"  Equal weights [0.5, 0.5]: index {ws_equal.best_index}, point {ws_equal.best_point}")
print(f"  f1-focused [0.8, 0.2]:    index {ws_f1.best_index}, point {ws_f1.best_point}")
print(f"  f2-focused [0.2, 0.8]:    index {ws_f2.best_index}, point {ws_f2.best_point}")

In [ ]:
# Visualize weighted sum selections
fig, ax = plt.subplots(figsize=(9, 6))

ax.scatter(F[:, 0], F[:, 1], s=30, alpha=0.5, c='lightgray', label='Pareto front')

selections = [
    (ws_equal, 'Equal [0.5, 0.5]', 'green', 'o'),
    (ws_f1, 'f1-focused [0.8, 0.2]', 'blue', 's'),
    (ws_f2, 'f2-focused [0.2, 0.8]', 'red', '^'),
]

for result, label, color, marker in selections:
    ax.scatter(result.best_point[0], result.best_point[1], 
               s=150, c=color, marker=marker, edgecolors='black', linewidth=2,
               label=label, zorder=5)

ax.set_xlabel("f1")
ax.set_ylabel("f2")
ax.set_title("Weighted Sum: Different Weight Choices")
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Tchebycheff Method

Minimizes the maximum weighted deviation from a reference point (typically the ideal):

$$\text{score} = \max_i \left( w_i \cdot |f_i - z_i^*| \right)$$

**Use when:** You want balanced solutions that don't sacrifice any objective too much.

In [ ]:
# Tchebycheff with equal weights
tch_equal = tchebycheff_scores(F, weights_equal)

# With custom reference point (ideal point)
ideal = np.array([0.0, 0.0])
tch_ideal = tchebycheff_scores(F, weights_equal, reference=ideal)

print("Tchebycheff Results:")
print(f"  Auto reference: index {tch_equal.best_index}, point {tch_equal.best_point}")
print(f"  Ideal [0,0]:    index {tch_ideal.best_index}, point {tch_ideal.best_point}")

In [ ]:
# Compare Weighted Sum vs Tchebycheff
fig, ax = plt.subplots(figsize=(9, 6))

ax.scatter(F[:, 0], F[:, 1], s=30, alpha=0.5, c='lightgray', label='Pareto front')

ax.scatter(ws_equal.best_point[0], ws_equal.best_point[1], 
           s=150, c='green', marker='o', edgecolors='black', linewidth=2,
           label='Weighted Sum', zorder=5)

ax.scatter(tch_equal.best_point[0], tch_equal.best_point[1], 
           s=150, c='purple', marker='D', edgecolors='black', linewidth=2,
           label='Tchebycheff', zorder=5)

ax.set_xlabel("f1")
ax.set_ylabel("f2")
ax.set_title("Weighted Sum vs Tchebycheff (both equal weights)")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nNote: Tchebycheff tends to find more 'balanced' solutions,")
print("while weighted sum may find corner/extreme solutions.")

## 4. Reference Point Method

Finds the solution closest to a specified **aspiration point**:

$$\text{score} = \| \mathbf{f} - \mathbf{r} \|_2$$

**Use when:** You have specific target values for each objective.

In [ ]:
# Define aspiration points
aspirations = [
    np.array([0.3, 0.4]),  # Moderate targets
    np.array([0.1, 0.8]),  # Low f1, accept high f2
    np.array([0.5, 0.2]),  # Accept moderate f1, want low f2
]

ref_results = [reference_point_scores(F, asp) for asp in aspirations]

print("Reference Point Results:")
for asp, res in zip(aspirations, ref_results):
    print(f"  Target {asp} → Selected {res.best_point} (distance: {res.scores[res.best_index]:.4f})")

In [ ]:
# Visualize reference point selections
fig, ax = plt.subplots(figsize=(9, 6))

ax.scatter(F[:, 0], F[:, 1], s=30, alpha=0.5, c='lightgray', label='Pareto front')

colors = ['blue', 'green', 'red']
for i, (asp, res) in enumerate(zip(aspirations, ref_results)):
    # Plot aspiration point
    ax.scatter(asp[0], asp[1], s=100, c=colors[i], marker='*', 
               edgecolors='black', linewidth=1, zorder=4)
    # Plot selected solution
    ax.scatter(res.best_point[0], res.best_point[1], s=120, c=colors[i], 
               marker='o', edgecolors='black', linewidth=2, 
               label=f'Target {asp}', zorder=5)
    # Draw line from aspiration to selection
    ax.plot([asp[0], res.best_point[0]], [asp[1], res.best_point[1]], 
            c=colors[i], linestyle='--', alpha=0.5)

ax.set_xlabel("f1")
ax.set_ylabel("f2")
ax.set_title("Reference Point Method: Closest to Target")
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Knee Point Method

Identifies the point of **maximum curvature** on the Pareto front - the "best compromise" where improving one objective requires significant sacrifice in the other.

**Use when:** You want an automatic compromise without specifying weights or targets.

In [ ]:
# Find knee point
knee = knee_point_scores(F)

print(f"Knee Point Result:")
print(f"  Index: {knee.best_index}")
print(f"  Point: {knee.best_point}")

In [ ]:
# Visualize knee point
fig, ax = plt.subplots(figsize=(9, 6))

ax.scatter(F[:, 0], F[:, 1], s=30, alpha=0.5, c='lightgray', label='Pareto front')

# Highlight knee point
ax.scatter(knee.best_point[0], knee.best_point[1], 
           s=200, c='red', marker='*', edgecolors='black', linewidth=2,
           label=f'Knee Point {knee.best_point.round(3)}', zorder=5)

# Draw line from endpoints to show the "bend"
order = np.argsort(F[:, 0])
F_sorted = F[order]
ax.plot([F_sorted[0, 0], F_sorted[-1, 0]], [F_sorted[0, 1], F_sorted[-1, 1]], 
        'k--', alpha=0.4, label='Endpoint line')

ax.set_xlabel("f1")
ax.set_ylabel("f2")
ax.set_title("Knee Point: Maximum Curvature (Best Compromise)")
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Compare All Methods

In [ ]:
# Collect all method results
methods = {
    "Weighted Sum": ws_equal,
    "Tchebycheff": tch_equal,
    "Reference [0.3,0.4]": ref_results[0],
    "Knee Point": knee,
}

fig, ax = plt.subplots(figsize=(10, 7))

ax.scatter(F[:, 0], F[:, 1], s=30, alpha=0.4, c='lightgray', label='Pareto front')

markers = ['o', 'D', 's', '*']
colors = ['green', 'purple', 'blue', 'red']
sizes = [150, 150, 150, 250]

for (name, res), marker, color, size in zip(methods.items(), markers, colors, sizes):
    ax.scatter(res.best_point[0], res.best_point[1], 
               s=size, c=color, marker=marker, edgecolors='black', linewidth=2,
               label=f'{name}: {res.best_point.round(3)}', zorder=5)

ax.set_xlabel("f1")
ax.set_ylabel("f2")
ax.set_title("MCDM Method Comparison")
ax.legend(loc='upper right', fontsize=9)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Summary table
print("MCDM Selection Summary:")
print("=" * 60)
print(f"{'Method':<25} {'f1':>10} {'f2':>10} {'Index':>8}")
print("-" * 60)
for name, res in methods.items():
    print(f"{name:<25} {res.best_point[0]:>10.4f} {res.best_point[1]:>10.4f} {res.best_index:>8}")

## 7. Interactive Weight Exploration

In [ ]:
# Explore how weight changes affect selection
weight_values = np.linspace(0.1, 0.9, 9)
selected_points = []

for w1 in weight_values:
    weights = np.array([w1, 1 - w1])
    res = weighted_sum_scores(F, weights)
    selected_points.append(res.best_point)

selected_points = np.array(selected_points)

# Plot
fig, ax = plt.subplots(figsize=(9, 6))

ax.scatter(F[:, 0], F[:, 1], s=30, alpha=0.4, c='lightgray', label='Pareto front')

# Color by weight
scatter = ax.scatter(selected_points[:, 0], selected_points[:, 1], 
                     s=100, c=weight_values, cmap='coolwarm', 
                     edgecolors='black', linewidth=1, zorder=5)

cbar = plt.colorbar(scatter, ax=ax, label='Weight on f1')

ax.set_xlabel("f1")
ax.set_ylabel("f2")
ax.set_title("Weighted Sum: Effect of Changing Weights")
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Blue = low weight on f1 (prioritize f2)")
print("Red = high weight on f1 (prioritize f1)")

## 8. Practical Workflow

In [ ]:
def select_solution(F, method="knee", weights=None, reference=None):
    """
    Helper function to select a single solution from a Pareto front.
    
    Args:
        F: Pareto front (n_points, n_obj)
        method: 'knee', 'weighted_sum', 'tchebycheff', 'reference'
        weights: Required for weighted_sum and tchebycheff
        reference: Required for reference method
    
    Returns:
        MCDMResult with best solution
    """
    if method == "knee":
        return knee_point_scores(F)
    elif method == "weighted_sum":
        if weights is None:
            weights = np.ones(F.shape[1]) / F.shape[1]
        return weighted_sum_scores(F, weights)
    elif method == "tchebycheff":
        if weights is None:
            weights = np.ones(F.shape[1]) / F.shape[1]
        return tchebycheff_scores(F, weights, reference)
    elif method == "reference":
        if reference is None:
            raise ValueError("Reference point required for reference method")
        return reference_point_scores(F, reference)
    else:
        raise ValueError(f"Unknown method: {method}")

# Example usage
print("Example workflow:")
print("-" * 40)
result = select_solution(F, method="knee")
print(f"1. No preference → Knee point: {result.best_point}")

result = select_solution(F, method="weighted_sum", weights=[0.7, 0.3])
print(f"2. Prefer f1 → Weighted sum: {result.best_point}")

result = select_solution(F, method="reference", reference=[0.2, 0.5])
print(f"3. Target [0.2, 0.5] → Reference: {result.best_point}")

## Summary

**MCDM Method Selection Guide:**

| Scenario | Recommended Method |
|----------|-------------------|
| No preference, want automatic compromise | **Knee Point** |
| Known objective importance | **Weighted Sum** |
| Want balanced solution | **Tchebycheff** |
| Specific target values | **Reference Point** |

**Key Functions:**
```python
from vamos import weighted_sum_scores, tchebycheff_scores, reference_point_scores, knee_point_scores

result = weighted_sum_scores(F, weights)  # weights: array of importance
result = tchebycheff_scores(F, weights)   # balanced solution
result = reference_point_scores(F, ref)   # closest to target
result = knee_point_scores(F)             # automatic compromise (2D)

# Result contains:
result.best_index   # Index of selected solution
result.best_point   # Objective values of selected solution
result.scores       # Scores for all solutions
```